## Cleaning
Drop column with extera catigorical data or data that just give bias to prefered players: usage, college year, ht, num, pfr, year, rec rank, pick, pik drafted, gbpm, offensive gdbm, defensive gdbm, type, Recruit Rank

In [3]:
#Alisha 
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

pd.set_option('display.max_columns', 70)

file_path = 'trainingData/CollegeBasketballPlayers2009-2021.csv'
dfBasketball = pd.read_csv(file_path)

# Inspect some of the data
dfBasketball.head()

dfBasketball.info()

dfBasketballTraining = dfBasketball.drop(columns=['usg','yr','ht', 'num', 'pfr', 'year', 'Rec Rank', 'gbpm', 'ogbpm', 'dgbpm', 'type'])

dfBasketballTraining.info(verbose=True, show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61061 entries, 0 to 61060
Data columns (total 66 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   player_name                      61061 non-null  object 
 1   team                             61061 non-null  object 
 2   conf                             61061 non-null  object 
 3   GP                               61061 non-null  int64  
 4   Min_per                          61061 non-null  float64
 5   Ortg                             61061 non-null  float64
 6   usg                              61061 non-null  float64
 7   eFG                              61061 non-null  float64
 8   TS_per                           61061 non-null  float64
 9   ORB_per                          61061 non-null  float64
 10  DRB_per                          61061 non-null  float64
 11  AST_per                          61061 non-null  float64
 12  TO_per            

C:\Users\Steven\AppData\Local\Temp\ipykernel_17864\3130527685.py:10: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBasketball = pd.read_csv(file_path)


In [7]:
def get_training_data(dataframe, replacement, features = 13, multiplicity = 10):
    trainingCount = int(features*multiplicity/2)
    not_drafted = dataframe[dataframe['pick'].isnull()].sample(n=trainingCount, replace=replacement)
    drafted = dataframe[dataframe['pick'].notnull()].sample(n=trainingCount, replace=replacement)
    return pd.concat([not_drafted, drafted], ignore_index=True)

fill in missing values for dunks made

In [3]:
#Josh 

fill in missing values for dunks attempt

In [4]:
#Josh 

fill in missing values for midrange shots made

In [5]:
#Heather 

fill in missing values for midrange attempts

In [6]:
#Heather 

fill in missing values for rim shots made

In [7]:
#Alisha 

fill in missing values for rim shots attempted 

In [8]:
#Alisha 

check for players still missing over 15% of the data and drop 

In [3]:
#Steve
threshold = 15
missing_percentage = (dfBasketball.isnull().sum() / len(dfBasketball)) * 100
columns_to_drop = []

for column in dfBasketball.columns:
    if missing_percentage[column] > threshold:
        columns_to_drop.append(column)

dfBasketballTraining = dfBasketball.drop(columns=columns_to_drop)

check and fill any remaining nulls in the columns 

In [9]:
#Steve
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputed_numeric = imputer.fit_transform(dfBasketballTraining.select_dtypes(include=np.number))
dfBasketballTraining_filled = pd.DataFrame(imputed_numeric, columns=dfBasketballTraining.select_dtypes(include=np.number).columns)
string_columns = dfBasketball.select_dtypes(include='object').columns
dfBasketballTraining_filled = pd.concat([dfBasketball[string_columns], dfBasketballTraining_filled], axis=1)
print("DataFrame with missing values filled in:")
print(dfBasketballTraining_filled)

DataFrame with missing values filled in:
              player_name              team  conf  yr      ht   num type  \
0           DeAndrae Ross     South Alabama    SB  So   2-Jun   NaN  all   
1           Pooh Williams          Utah St.   WAC  So   4-Jun   NaN  all   
2           Jesus Verdejo     South Florida    BE  Sr   4-Jun   NaN  all   
3         Mike Hornbuckle        Pepperdine   WCC  Sr   4-Jun   NaN  all   
4           Anthony Brown           Pacific    BW  Sr   8-Jun   NaN  all   
...                   ...               ...   ...  ..     ...   ...  ...   
61056      Trey Patterson         Villanova    BE  Fr   8-Jun  12.0  all   
61057  Stavros Polatoglou  Northwestern St.  Slnd  Fr  11-Jun  33.0  all   
61058          Sandy Ryan            Tulane  Amer  Fr     NaN  24.0  all   
61059           Ty Larson        Texas Tech   B12  Fr     NaN  30.0  all   
61060         Jaden Jones           Rutgers   B10  Fr   8-Jun  45.0  all   

      Unnamed: 64    GP  Min_per   Ortg   usg 

## Transform 